<a href="https://colab.research.google.com/github/agnesedaff/Implicit_obj_completion/blob/main/IMPLICIT_prova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install spacy
!python -m spacy download it_core_news_lg
!python -m pip install transformers

2024-01-23 16:00:05.079754: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 16:00:05.079800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 16:00:05.081103: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 16:00:05.088806: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 16:00:06.529326: W tensorflow/compiler/tf2

In [2]:
import spacy
nlp = spacy.load("it_core_news_lg")

In [3]:
from transformers import pipeline
# Initialize the fill-mask pipeline with the Italian model
unmasker = pipeline('fill-mask', model="dbmdz/bert-base-italian-xxl-cased", tokenizer = "dbmdz/bert-base-italian-xxl-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

Prova su 8 frasi rappresentative (1 MASK)

In [4]:
#creiamo una variabile con le frasi di prova

sentences = ["Mi piace bere il vino. Quando bevo [MASK] mi sento bene.",
             "Mi piace bere l'acqua. Quando bevo [MASK] mi sento bene.",
             "Mi piace bere succhi di frutta. Quando bevo [MASK] mi sento bene.",
             "Mi piace bere i succhi di frutta, l'acqua e il vino. Quando bevo [MASK] mi sento bene",
             "Versò la minestra e gliela diede da mangiare. Lui mangiò [MASK] e ringraziò",
             "Versò la minestra e gliela diede da bere. Lui bevve [MASK] e ringraziò",
             "Versò la pozione e gliela diede da bere. Lui bevve [MASK] e ringraziò",
             "Versò la pozione verde e gliela diede da bere. Lui bevve [MASK] verde e ringraziò"]

In [5]:
for sentence in sentences:
    noun_predictions = []
    results = unmasker(sentence, top_k=10) #usiamo l'unmasker per ottenere i primi 10 risultati
    for prediction in results: #per ogni risultato, applichiamo il pos tagging di SpaCy in modo da isolare solo i nomi
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction) #aggiungiamo i nomi candidati alla lista noun_predictions
    if noun_predictions:
        #ordiniamo i candidati in base al prediction score
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        #isoliamo il candidato con lo score più alto e stampiamo il punteggio e la frase completa
        print(noun_predictions[0]["token_str"], noun_predictions[0]["score"], noun_predictions[0]["sequence"])

    else:
        print("unknown")

vino 0.7271541953086853 Mi piace bere il vino. Quando bevo vino mi sento bene.
acqua 0.5717719197273254 Mi piace bere l'acqua. Quando bevo acqua mi sento bene.
succhi 0.07958560436964035 Mi piace bere succhi di frutta. Quando bevo succhi mi sento bene.
alcol 0.053784724324941635 Mi piace bere i succhi di frutta, l'acqua e il vino. Quando bevo alcol mi sento bene
unknown
acqua 0.03916392847895622 Versò la minestra e gliela diede da bere. Lui bevve acqua e ringraziò
acqua 0.022243786603212357 Versò la pozione e gliela diede da bere. Lui bevve acqua e ringraziò
acqua 0.1458343118429184 Versò la pozione verde e gliela diede da bere. Lui bevve acqua verde e ringraziò


Prova su 80 frasi dell'IMPLICIT dataset, primi 4 verbi (2 MASK)

In [6]:
# Define a list of sentences
sentences = ["Beh, in questo caso credo la cosa migliore per un musicista coinvolto sia semplicemente quella di ascoltare [MASK][MASK] e ringraziare per il complimento...",
"La Ecm è la casa discografica di Monaco di Baviera famosa per le splendide copertine raffiguranti paesaggi nordici e per la pulizia del suono che fa dire ai musicofili: ecco, ascoltate [MASK][MASK], questo è il suono del silenzio.",
"durante il monologo U. rimane impietrito ad ascoltare [MASK][MASK], terrorizzato",
"Un quarto aspetto di diversità è: che cosa lo studente deve fare: deve riflettere a lungo, deve parlare, deve prendere appunti, deve disegnare, deve leggere, deve completare un grafico, deve ascoltare [MASK][MASK], ecc.",
"Semplicemente perchè non ho mai fatto un'esperienza di concerti dove il pubblico ascolta [MASK][MASK] con attenzione.",
"I brani, caratterizzati da sonorità pop rock di matrice inglese, ma con una sensibilità italiana nella scrittura dei testi e nella composizione dele melodie, hanno lo scopo di entrare in risonanza con le corde più profonde di chi ascolta [MASK][MASK].",
"Ascoltando [MASK][MASK] qua e là, senza dare troppo nell'occhio, scoprii che non ero nell'anticamera del Paradiso, ma il mio shock non fu minore.",
"Maurizia guardava e ascoltava [MASK][MASK] sorniona, girando da un gruppo all'altro.",
"Le canticchiavo in un inglese totalmente inventato, qualche volta i brani accompagnavano il monoscopio della TV ed io mi fermavo trasognato ad ascoltare [MASK][MASK], guardando fisso lo schermo dove non succedeva assolutamente nulla.",
"Il corso intende formare una figura professionale in grado di cogliere l'essenza di un testo elementare di inglese, possedere abilità ricettive (ascoltare [MASK][MASK] e leggere ad un livello elementare, legate al lavoro in ufficio).",
"Il senatore a vita ascolta [MASK][MASK] e si rigetta nelle sue carte.",
"Procedeva cauto, sostando di frequente per ascoltare [MASK][MASK] e non riprendendo la marcia se non quando era ben certo di non aver dinanzi il nemico.",
"Osservate il mio bel mantello: se ascoltate con attenzione [MASK][MASK] udirete il canto, che di ogni donna è l'essenza, prezioso se lo si sa incontrare.",
"Renzo le raccontò brevemente la storia di quella mattina: ella ascoltava [MASK][MASK] con angoscia: e quando udì il nome di don Rodrigo, -ah!",
"Vai in WGPlayer ed ascolta [MASK][MASK] con un suono un po'decente.",
"Hang appoggiò un orecchio sul petto di Romero ed ascoltò [MASK][MASK] con profondo raccoglimento.",
"Se c'è un pianista che vale la pena di ascoltare, insomma, qualsiasi cosa voglia suonare, ebbene è Keith Jarrett, capace di commuovere e di esaltare, in grado di inventare inedite figure musicali ed al tempo stesso in grado di far viaggiare chi ascolta [MASK][MASK] in una straordinaria macchina sonora che non conosce limiti di tempo e di spazio.",
"Chi canta a squarciagola, chi ascolta [MASK][MASK] in silenzio.",
"Vittorio Messori ascolta [MASK][MASK] in silenzio, alla cornetta arrivano sospiri perplessi sull'apertura al rock del Vaticano.",
"Il gusto di stare fermi a contemplare, ad ascoltare [MASK][MASK] in silenzio, lasciando che la musica di luoghi possibilmente solitari entri piano piano nell'anima e risintonizzi il pensiero su un'altra onda, un'altra percezione, un'altra intuizione.",
"La risposta alla vostra domanda verrà pubblicata comunque: si tratta solo di attendere [MASK][MASK] un po'.",
"A seguito della riforma del CNR e del conseguente commissariamento, la Commissione decise peraltro di attendere [MASK][MASK] prima di ascoltare il professor De Maio onde consentirgli di avviare la fase di transizione.",
"Bisogna attendere [MASK][MASK] ancora.",
"La festa deve iniziare; il mio popolo attende [MASK][MASK] da troppo tempo.",
"Questo risultato conferma uno studio precedente di Ramsay, Barbesi e Preece dove utenti che attendevano [MASK][MASK] più di 41 secondi giudicavano negativamente anche il contenuto delle pagine, definendolo addirittura meno interessante (ovviamente a parità di contenuto) rispetto a quelle che si caricavano più velocemente, oltre che più difficile da scorrere!",
"Ai tifosi nel frattempo non resta che attendere [MASK][MASK] con ansia e dare fiducia a Naldi che qualche giorno fa ha detto: 'A maggio pagherò tutti'.",
"Io, fortunatamente, appartengo alla prima categoria: mi rassegno all'idea che bisogna attendere [MASK][MASK] ancora un po'e poi, ringraziando il buon Dio, arriverà anche il mio turno per le agognate ferie.",
"Attendete [MASK][MASK]: adesso vedrò.",
"Poi i riflettori si riaccendevano e Pesce riprendeva a recitare, per la gioia del pubblico che aveva atteso [MASK][MASK] pazientemente per rivederlo in azione: pur nella sventura, una conferma della sua bravura.",
"Per sapere se il carburante finirà per aumentare comunque, bisognerà attendere [MASK][MASK] qualche settimana.",
"Gli agenti hanno deciso di attendere [MASK][MASK] in zona e dopo un'ora si è avvicinata una persona che ha riposizionato le trappole collocando nuove esche.",
"Ecco i dettagli sulla morte di Padme: Un droide fluttuante si avvicina ai due Jedi (Yoda e Obi-Wan) che attendono [MASK][MASK] in ansia, il suo volto metallico non comunica espressioni, ma i movimenti delle braccia e del corpo manifestano il suo stato d'animo, sullo sfondo vediamo Padmé su un lettino ovale fluttuante, è morta, coperta fino al petto da un lenzuolo bianco, indossa una veste azzurra, attorno a lei si muovono le grandi mani di medici alieni blu, non c'è nulla da fare.",
"Vecchi amici e avversari attendono [MASK][MASK] in luoghi inaspettati, i rischi trascendentali abbondano, e il salvatore più improbabile del multiverso incontra meraviglie e pericoli sufficienti a fare ammutolire lo stesso Q. O quasi.",
"Usufruendo di questa possibilità, il cittadino non dovrà attendere [MASK][MASK] in fila e potrà ottenere immediatamente il proprio duplicato.",
"Sopraggiunse l'abate che fu lasciato solo con l'ammalata e Cassandrino attese [MASK][MASK] in un gabinetto attiguo.",
"Drin, drin, una voce asettica rispose: 'La prego di attendere [MASK][MASK] in linea', dopo di che partì una strana musichetta 'I sogni son desideri...'",
"Ormai è più che decente riattaccare, evidentemente non c'è nessuno o, se qualcuno c'è, ha fatto la sua scelta; ma adesso che si è atteso [MASK][MASK] così a lungo tanto vale andare oltre, sarà più profonda la certezza di avere compiuto il proprio dovere fino in fondo, ancora un ultimo squillo, adesso può davvero rinunciare, non c'è nessuno, sono in vacanza.",
"Si trattava di un caso terribile, un ragazzo quindicenne che attendeva [MASK][MASK] lucidamente per quattro ore, informato dai medici di ciò, lo spengersi della propria esistenza, in seguito ad un incidente stradale.",
"Ben presto il nostro settore si riempie mentre il resto dello stadio è ancora semdieserto; durante la partita il tifo da parte nostra è discreto e anche la partita termina positivamente; alla fine, dopo la solita attesa dentro lo stadio, siamo costretti ad attendere [MASK][MASK] per un'altra ora abbondante nel piazzale prima di ripartire, perché il treno del ritorno parte verso le otto.",
"Qualora a seguito della verifica dovesse accertarsi la mancanza del numero legale, si attende [MASK][MASK] per una durata massima di quindici minuti, dopodiché, se la situazione dovesse persistere, la seduta viene sciolta.",
"(Beve [MASK][MASK] direttamente dalla bottiglia) Veronica - Guardi che è un cognac francese!",
"Quand'ebbe finito di dargli da bere [MASK][MASK], disse: Io ne attingerò anche per i tuoi cammelli finchè abbiano bevuto a sufficienza.",
"E'quindi della massima importanza adottare opportune contromisure: svolgere una regolare attività fisica, aumentare il consumo di alimenti ricchi di fibre, bere [MASK][MASK] regolarmente, evitare i grassi animali privilegiando quelli vegetali da consumare preferibilmente crudi.",
"La giovinezza, l'ardore, il dolore di Giacinto si trasfondevano in lei: dimenticava i suoi anni, il suo aspetto, la sua assenza; le sembrava d'essere distesa sotto un'acqua limpida nel folto di un bosco e di vedere una figura curvarsi a bere, a bere [MASK][MASK], sopra la sua bocca: era Giacinto, ma era anche lei, Noemi viva, assetata d'amore: era uno spirito misterioso che sorbiva tutta l'acqua della sorgente, tutta la vita dalla bocca di lei, tanta sete insaziabile aveva; e si stendeva poi nel cavo",
"Da quel 26 dicembre non vuole più bere [MASK][MASK] nè lavarsi.",
"L'Imperatore fece roteare il vino nella coppa rimanendo ad osservare le sfumature del perfetto color rubino, bevve [MASK][MASK], strizzando gli occhi nell'assaporare l'immediato calore che dalla cavità orale si diffondeva giù fino allo stomaco, con una sensazione piacevolmente violenta, e pensò che nessun essere umano gli aveva mai trasmesso un calore così intenso.",
"E poi, con magistrale lentezza, la creatura gli morse il braccio e come il sangue iniziò a fluire, il Denizen bevve [MASK][MASK].",
"Sollevò la tazzina fino alla bocca e bevve [MASK][MASK]: un gesto di copertura automatico, il caffè doveva essere gelato.",
"Spike bevve [MASK][MASK] con una punta di diffidenza (non beveva sangue da un bel pezzo sottoterra) e si sorprese più affamato di quanto non avesse immaginato, ripulendo ben bene la tazza e rendendola lucida.",
"lei prese il vasetto, lo guardò un attimo in controluce e decise di non pensarci, versò il caffè tiepido e bevve [MASK][MASK], sentì alcuni filamenti entrarle in bocca e finirle in gola e tenne a stento un conato di vomito, poi versò il caffè a suo fratello e si stese sulla poltrona da dentista del nonno.",
"E di nuovo gli porsi vino lucente; tre volte gliene porsi, tre volte bevve [MASK][MASK], da pazzo.",
"Nessun bicchiere pulito e bevve [MASK][MASK] dal rubinetto, tanto aveva sentito che ultimamente l'acqua pubblica era più sicura della minerale.",
"Gliene servì ed egli mangiò, gli portò il vino ed egli bevve [MASK][MASK].",
"Giacobbe diede ad Esaù il pane e la minestra di lenticchie; questi mangiò e bevve [MASK][MASK], poi si alzò e se ne andò.",
"Si pronuncia Ateneo sul Ciclo di Eracle: 'E Stesicoro nomina la coppa usata nella caverna del centauro Pholus come skyphion depas, cioè 'simile a skyphos'; di Eracle egli dice: 'E afferrando la coppa simile a skyphos, di misura uguale a tre lagynos, egli l'avvicinò alle sue labbra, e bevve [MASK][MASK].",
"Si trascinò affamato verso il ruscello e bevve [MASK][MASK] come un animale.",
"stava tornando a casa dopo aver passato la serata al bar a bere [MASK][MASK]...",
"L'anno scorso gli inquirenti avevano cercato le impronte del ragazzo su alcune bottiglie di vino trovate nel ranch, al fine di sostenere le accuse secondo le quali Jackson avrebbe costretto il giovane a bere [MASK][MASK] per poi violentarlo.",
"Le ricerche dicono che i ragazzi bevono [MASK][MASK] per essere più socievoli, disinibiti ed euforici.",
"Il nostro obiettivo invece è quello di spiegare ai giovani che per divertirsi non è necessario ubriacarsi e soprattutto far capire loro che, dopo aver bevuto [MASK][MASK], è pericoloso mettersi al volante.",
"Naturalmente i brani sono stati arrangiati appositamente per lei, che oltre a cantare [MASK][MASK] introduce le canzoni con delle parti recitate in cui da il meglio di se stessa",
"La conclusione, dopo un'altra uscita di scena, è affidata a un altro pezzo storico, 'Feel', eseguito con Bent alla chitarra e voce e Snah e Gebhardt a sostituire il mellotron con dei coretti più teneri che altro; e tutto il pubblico a cantare [MASK][MASK] all'unisono, a ripetere il miracolo che da dieci anni questo gruppo mette in atto sui palchi di tutta Europa: senza una copertura pubblicitaria degna di tal nome, contando praticamente solo sul passaparola, i Motorpsycho sono in grado di raccogliere",
"Poi lui smetteva di cantare [MASK][MASK] per rispondere al suo gesto e giuro che mi sembrò di vedere i due becchi sfiorarsi in un fuggevole bacio.",
"Infine elogio i ragazzi della nord che cantano [MASK][MASK] incitano ma sono spesso poco seguiti (in nord ci sono molte persone in età avanzata che non hanno voglia di cantare ma da loro è capibile) e mi ritorna in mente una lettera di un ragazzo che si lamentava che i biglietti della Sud per il derby fossero dati anche a quelli della Nord, dicendo che la Sud è roba loro.",
"Stavolta però in qualsiasi angolo della Curva si è cantato [MASK][MASK] e l'effetto canoro e coreografico hanno dimostrato ancora una volta che il nostro tifo è al di sopra di tanti altri che professano i migliori del mondo.",
"Mi si chiedeva sempre di cantare [MASK][MASK] per i parenti",
"Nel corso della sua carriera, cantò [MASK][MASK] con la celebre Maria Callas che addirittura gli chiese il bis dell'esecuzione.",
"Cantò [MASK][MASK] per noi, anticipando il suo intervento serale in una piazza gremita.",
"Nel 1999 è invitato a Milano, ad esibirsi nella serata tributo a Jeff Buckley: lì cantò [MASK][MASK] insieme a Morgan dei Bluvertigo e Cristina Donà.",
"O, se è stata, chissà quando, chissà dove: al tempo di Gilgamesh, magari, quando sulle mura di Ninive qualcuno cantava [MASK][MASK] alla luna convinto di aver liberato il mondo.",
"Quando i suoi compagni lo stavano per buttare infine nelle acque ruggenti lui disse: lasciatemi solo per un po', andò alla poppa della nave e cominciò a cantare una semplice canzone popolare irlandese ma così bella e armoniosa era la sua voce che la sirena ne rimase incantata e mentre lui cantava [MASK][MASK] lei assunse un espressione assorta e malinconica ed anche il mare si calmò con lei.",
"non so se fosse una rielaborazione dei Sinitah (era Yamil che cantava [MASK][MASK]), a me piace pensare che quello fosse il sapore originale che aveva in Africa, prima che attraversasse l'oceano verso cuba insieme ai neri schiavi.",
"Trovò Zhuang-zi seduto a gambe incrociate a forma di setaccio, che cantava [MASK][MASK] battendo il tempo su una scodella.",
"A cinque anni, cantava [MASK][MASK] a due voci con il nonno, direttore del coro di Monfalcone, e a undici scriveva la sua prima canzone.",
"La ragazza che aveva già sedici anni, seduta all'ombra di un albero, cantava [MASK][MASK] tristamente.",
"Achille era già uomo, ma ancora temeva la verga, quando cantava [MASK][MASK] sui monti della sua patria, e mai avrebbe riso della coda che aveva il citaredo suo maestro; oggi i discepoli pigliano a bastonate Rufo e gli altri maestri; Rufo, che pure è spesso detto il Cicerone allòbrogo.",
"Era nata il 23 maggio 1928 a Maysville, Kentucky, Usa, e ancora ragazzina, cantava [MASK][MASK] con sua sorella Betty, alle riunioni politiche a sostegno del nonno paterno.",
"L'amico Vladimir da giovane faceva l'agente segreto per il Kgb, mentre Berlusconi da ragazzo cantava [MASK][MASK] sulle navi da crociera.",
"Riguardo il tizio che voleva picchiare il Reverendo perché non cantava [MASK][MASK]: capisco l'entusiasmo, il tifo, e tanto altro.",
"Manco a dirlo cantava [MASK][MASK] solo la Nord."]

Prova 0:
1.   fornire una lista di Determiners
2.   iterare su questi per completare il Noun
3.   prendere la combinazione con punteggio più alto

In [7]:
output = []

for sentence in sentences:

    noun_predictions = [] #1
    det_predictions = ["il ", "lo ", "la ", "i ", "gli ", "le ", ""]
    for det in det_predictions:
        new_sentence = sentence.replace("[MASK]", det, 1)
        results = unmasker(new_sentence, top_k=10)

        for prediction in results: #2
            doc = nlp(prediction["token_str"])
            for token in doc:
                if token.pos_ == "NOUN":
                    noun_predictions.append(prediction)

    if noun_predictions: #3
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output.append(noun_predictions[0]["token_str"])
        print(noun_predictions[0]["token_str"], noun_predictions[0]["score"], noun_predictions[0]["sequence"])
    else:
        output.append("unknown")
        print("unknown")

print(output)

canzoni 0.47544559836387634 Beh, in questo caso credo la cosa migliore per un musicista coinvolto sia semplicemente quella di ascoltare le canzoni e ringraziare per il complimento...
silenzio 0.8867990970611572 La Ecm è la casa discografica di Monaco di Baviera famosa per le splendide copertine raffiguranti paesaggi nordici e per la pulizia del suono che fa dire ai musicofili : ecco, ascoltate il silenzio, questo è il suono del silenzio.
parole 0.47577404975891113 durante il monologo U. rimane impietrito ad ascoltare le parole, terrorizzato
musica 0.9393972754478455 Un quarto aspetto di diversità è : che cosa lo studente deve fare : deve riflettere a lungo, deve parlare, deve prendere appunti, deve disegnare, deve leggere, deve completare un grafico, deve ascoltare la musica, ecc.
musica 0.9581058025360107 Semplicemente perchè non ho mai fatto un'esperienza di concerti dove il pubblico ascolta la musica con attenzione.
strumento 0.8888481855392456 I brani, caratterizzati da sonorità po

prova 1:
1.   riempire il primo [MASK] con un Determiner
2.   creare una nuova frase con il primo [MASK] completato
3.   completare la nuova frase con il Noun con il punteggio più alto

In [8]:
output1 = []

for sentence in sentences:
    results = unmasker(sentence, top_k=10)

    det_predictions = [] #1
    for prediction in results[0]:
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "DET":
                det_predictions.append(prediction)
    if det_predictions:
            det_predictions.sort(key=lambda x: x["score"], reverse=True)
            determiner = det_predictions[0]["token_str"]

            new_sentence = sentence.replace("[MASK]", determiner, 1) #2
    else:
            new_sentence = sentence.replace("[MASK]", "", 1)

    results_new = unmasker(new_sentence, top_k=10) #3
    noun_predictions = []
    for prediction in results_new:
        doc = nlp(prediction['token_str'])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction)
    if noun_predictions:
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output1.append(noun_predictions[0]["token_str"])
        print(noun_predictions[0]["token_str"], noun_predictions[0]["score"], noun_predictions[0]["sequence"])
    else:
        output1.append("unknown")
        print("unknown")

print(output1)

brano 0.28247228264808655 Beh, in questo caso credo la cosa migliore per un musicista coinvolto sia semplicemente quella di ascoltare il brano e ringraziare per il complimento...
silenzio 0.8867990970611572 La Ecm è la casa discografica di Monaco di Baviera famosa per le splendide copertine raffiguranti paesaggi nordici e per la pulizia del suono che fa dire ai musicofili : ecco, ascoltate il silenzio, questo è il suono del silenzio.
musica 0.30218642950057983 durante il monologo U. rimane impietrito ad ascoltare la musica, terrorizzato
brano 0.2704172432422638 Un quarto aspetto di diversità è : che cosa lo studente deve fare : deve riflettere a lungo, deve parlare, deve prendere appunti, deve disegnare, deve leggere, deve completare un grafico, deve ascoltare un brano, ecc.
concerto 0.19153504073619843 Semplicemente perchè non ho mai fatto un'esperienza di concerti dove il pubblico ascolta il concerto con attenzione.
brano 0.47208166122436523 I brani, caratterizzati da sonorità pop ro

prova 2:
1.   riempire il secondo [MASK] ignorando il primo



In [11]:
import re
output2 = []

for sentence in sentences:
    results = unmasker(sentence, top_k=10)

    noun_predictions = [] #1
    for prediction in results[1]:
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction)
    if noun_predictions:
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output2.append(noun_predictions[0]["token_str"])
        sequence = noun_predictions[0]["sequence"]
        sequence = re.sub("\[CLS\]","", sequence)
        sequence = re.sub("\[SEP\]","", sequence)
        print(noun_predictions[0]["token_str"], noun_predictions[0]["score"], sequence)
    else:
        output2.append("unknown")
        print("unknown")

print(output2)

musica 0.12115839868783951  Beh, in questo caso credo la cosa migliore per un musicista coinvolto sia semplicemente quella di ascoltare [MASK] musica e ringraziare per il complimento... 
musica 0.11783631891012192  La Ecm è la casa discografica di Monaco di Baviera famosa per le splendide copertine raffiguranti paesaggi nordici e per la pulizia del suono che fa dire ai musicofili : ecco, ascoltate [MASK] musica, questo è il suono del silenzio. 
musica 0.07855569571256638  durante il monologo U. rimane impietrito ad ascoltare [MASK] musica, terrorizzato 
musica 0.49777165055274963  Un quarto aspetto di diversità è : che cosa lo studente deve fare : deve riflettere a lungo, deve parlare, deve prendere appunti, deve disegnare, deve leggere, deve completare un grafico, deve ascoltare [MASK] musica, ecc. 
musica 0.10397666692733765  Semplicemente perchè non ho mai fatto un'esperienza di concerti dove il pubblico ascolta [MASK] musica con attenzione. 
musica 0.37774932384490967  I brani, car

Secondo step -- creazione di un **dizionario** e **calcolo** **della** **similarity**

In [12]:
#prova similarity score

doc1 = nlp("Gardaland")
doc2 = nlp("verbo")

if doc1.has_vector and doc2.has_vector:
  print(doc1.has_vector)
  print(doc2.has_vector)
  similarity = doc1.similarity(doc2)
  similarity_rounded = round(similarity, 2)
  print(similarity_rounded)
else:
    print("no vector found")

True
True
-0.14


In [13]:
# Per calcolare la similarity tra le parole predicted e l'output del modello:
# Creiamo un dizionario che associa le coppie dell'una e dell'altra lista
# Per evitare sovrapposizioni, aggiungiamo una numerazione alle parole ripetute

predicted = ["parola","musica","discorso","conversazione","concerto","brano","suono","suono","brano","conversazione","discorso","rumore","suono","storia","musica","respiro","musica","canzone","radio","suono","risposta","sviluppo","sviluppo","evento","caricamento","conferma","turno","risposta","arrivo","sviluppo","persona","notizia","arrivo","turno","fine","turno","risposta","esito","treno","soluzione","vino","acqua","acqua","acqua","acqua","vino","sangue","caffè","sangue","caffè","vino","acqua","vino","minestra","vino","acqua","alcol","vino","alcolico","alcolico","canzone","pezzo","melodia","coro","coro","canzone","brano","canzone","brano","inno","canzone","canzone","canzone","canzone","canto","canzone","canzone","brano","canto","coro"]

word_dict = {}
counter = {}

for predicted_word, output_word in zip(predicted, output):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict[new_key] = output_word

print("word_dict: ", word_dict)
print(len(word_dict))

word_dict1 = {}
counter = {}

for predicted_word, output_word in zip(predicted, output1):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict1[new_key] = output_word

print("word_dict1: ", word_dict1)
print(len(word_dict1))

word_dict2 = {}
counter = {}

for predicted_word, output_word in zip(predicted, output2):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict2[new_key] = output_word

print("word_dict2: ", word_dict2)
print(len(word_dict2))

word_dict:  {'parola': 'canzoni', 'musica': 'silenzio', 'discorso': 'parole', 'conversazione': 'musica', 'concerto': 'musica', 'brano': 'strumento', 'suono': 'musica', 'suono_1': 'spettacolo', 'brano_1': 'musica', 'conversazione_1': 'parole', 'discorso_1': 'giudici', 'rumore': 'notizie', 'suono_2': 'sguardo', 'storia': 'spettacolo', 'musica_1': 'musica', 'respiro': 'parole', 'musica_2': 'strumento', 'canzone': 'musica', 'radio': 'spettacolo', 'suono_3': 'musica', 'risposta': 'risposte', 'sviluppo': 'atti', 'sviluppo_1': 'risultati', 'evento': 'elezioni', 'caricamento': 'schermo', 'conferma': 'stadio', 'turno': 'ferie', 'risposta_1': 'risposta', 'arrivo': 'spettacolo', 'sviluppo_2': 'momento', 'persona': 'sbarco', 'notizia': 'Jedi', 'arrivo_1': 'ritorno', 'turno_1': 'messa', 'fine': 'ospiti', 'turno_2': 'telefonate', 'risposta_2': 'cose', 'esito': 'esami', 'treno': 'stadio', 'soluzione': 'sedute', 'vino': 'champagne', 'acqua': 'sera', 'acqua_1': 'alcolici', 'acqua_2': 'vino', 'acqua_3':

In [15]:
# Calcoliamo la similarity tra le parole del dizionario con SpaCy.
# Per fare questo dobbiamo sistematicamente rimuovere il numero di indice aggiunto prima.

import re

# Funzione per calcolare la similarity tra due parole
def calculate_similarity(word1, word2):
    word1 = re.sub(r"_\d+", "", word1)
    word2 = re.sub(r"_\d+", "", word2)
    word1_token = nlp(word1)
    word2_token = nlp(word2)
    if word1_token.has_vector and word2_token.has_vector:
        similarity = round(word1_token.similarity(word2_token),2)
        return similarity

# Calcola la similarity per ogni coppia nel dizionario

similarity_scores = []
for key, value in word_dict.items():
    similarity = calculate_similarity(key, value)
    similarity_scores.append(similarity)
print(similarity_scores)
print("media esperimento 0 =", round(sum(similarity_scores) / len(similarity_scores),2))

similarity_scores1 = []
for key, value in word_dict1.items():
    similarity = calculate_similarity(key, value)
    similarity_scores1.append(similarity)
print(similarity_scores1)
print("media esperimento 1 =", round(sum(similarity_scores1) / len(similarity_scores1),2))

similarity_scores2 = []
for key, value in word_dict2.items():
    similarity = calculate_similarity(key, value)
    similarity_scores2.append(similarity)
print(similarity_scores2)
print("media esperimento 2 =", round(sum(similarity_scores2) / len(similarity_scores2),2))

[0.24, 0.2, 0.33, 0.38, 0.5, 0.18, 0.5, 0.33, 0.45, 0.32, 0.3, 0.07, 0.37, 0.36, 1.0, 0.22, 0.19, 0.62, 0.12, 0.5, 0.66, 0.17, 0.35, 0.22, 0.32, 0.08, 0.19, 1.0, 0.22, 0.29, -0.1, 0.04, 0.65, 0.04, 0.05, 0.14, 0.17, 0.24, 0.23, 0.17, 0.58, 0.1, 0.31, 0.3, 0.31, 1.0, 0.48, 1.0, 0.17, 1.0, 1.0, 1.0, 0.2, 0.22, 0.24, 0.56, 0.22, 0.39, 0.64, 0.34, 0.35, 0.52, 0.72, 0.27, 0.18, 0.7, 0.13, 0.7, 0.75, 0.18, 1.0, 1.0, 0.7, 0.7, 0.42, 0.32, 0.7, 0.62, 0.42, 0.08]
media esperimento 0 = 0.41
[0.23, 0.2, 0.17, 0.14, 1.0, 1.0, 0.51, 0.5, 0.62, 0.46, 0.37, 0.24, 0.34, 0.33, 0.27, 0.4, 0.54, 0.1, 0.42, 0.4, 0.19, -0.02, -0.02, 0.5, 0.07, 0.28, 0.13, 0.19, 0.1, -0.01, 0.24, 0.29, 0.65, 0.46, 0.12, 0.29, 0.28, -0.03, 1.0, 0.62, 1.0, 0.3, 1.0, -0.06, 0.31, 1.0, 1.0, 1.0, 0.35, 1.0, 1.0, 0.3, 1.0, 0.3, 0.24, 0.3, 0.32, 1.0, 0.34, 0.37, 0.7, 1.0, 0.6, -0.07, 1.0, 0.75, 1.0, 0.37, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.42, 1.0, 1.0, 0.56, 0.42, 0.31]
media esperimento 1 = 0.51
[0.32, 1.0, 0.17, 0.38, 0.5, 0.45, 0